In [1]:
# make sure run prepare-malay-stt-train.ipynb first

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/imda/part3-same-splitted.tar
# !tar -xf part3-same-splitted.tar

In [3]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/imda/TTS.zip
# !unzip TTS.zip

In [4]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/imda/part1.tar
# !tar -xf part1.tar
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/imda/SCRIPT.zip
# !unzip -o SCRIPT.zip -d WAVE-text
# !rm SCRIPT.zip

In [5]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [6]:
import pandas as pd
from glob import glob
from tqdm import tqdm
from unidecode import unidecode
import random
import json

In [7]:
base_directory = '/home/husein/speech-bahasa'

In [21]:
with open(f'{base_directory}/CHANNEL0/SCRIPT/FEMALE_01.txt') as fopen:
    texts = fopen.read().split('\n')

singlishs = []
for t in texts:
    splitted = t.split('\t')
    if len(splitted) == 2:
        singlishs.append((f'{base_directory}/CHANNEL0/WAVE/FEMALE_01/{splitted[0]}.wav', splitted[1]))
        
len(singlishs)

6033

In [22]:
wave_texts = glob('WAVE-text/*.TXT')

for f in tqdm(wave_texts):
    speaker = f.split('/')[1].replace('.TXT', '')
    channel = speaker[-1]
    speaker = speaker[1:-1]
    
    with open(f) as fopen:
        texts = list(filter(None, fopen.read().split('\n')))
        texts = [texts[i: i + 2] for i in range(0, len(texts), 2)]
    
    for text in texts:
        splitted = text[0].split('\t')
        wav = unidecode(splitted[0])
        t = text[1].split('\t')[1]
        path = f'{base_directory}/WAVE/SPEAKER{speaker}/SESSION{channel}/{wav}.WAV'
        
        if os.path.exists(path) and len(t):
            singlishs.append((path, t))
        else:
            print(splitted, path)

100%|██████████| 2034/2034 [00:04<00:00, 415.44it/s]


In [25]:
len(singlishs)

762375

In [26]:
singlish = glob(f'{base_directory}/part3-splitted/wav/*.wav')
len(singlish)

1261328

In [27]:
def get_after_mandarin(word):
    if '<mandarin>' in word:
        w = word.split('>')[1].split(':')[1]
        return w.split('</')[0]
    else:
        return word
    
def get_before_mandarin(word):
    if '</mandarin>' in word:
        return word.split('</')[0]
    else:
        return word

def replace_paralinguistic(string, replaces = ['(ppb)', '(ppc)', '(ppl)', '(ppo)', '<UNK>', '<MANDARIN>']):
    for r in replaces:
        string = string.replace(r, ' ')
    string = string.split()
    string = [get_after_mandarin(w) for w in string]
    string = [get_before_mandarin(w) for w in string]
    string = [w for w in string if w[0] not in '<[(' and w[-1] not in '>])']
    return ' '.join(string)

for i in tqdm(singlish):
    try:
        p = i.replace('/wav','/text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        if len(text) < 2:
            continue
        if text[0] == '<' and text[-1] == '>':
            continue
        before = text
        text = replace_paralinguistic(text)
        singlishs.append((i, text))
    except Exception as e:
        pass

100%|██████████| 1261328/1261328 [01:20<00:00, 15617.53it/s]


In [28]:
from sklearn.model_selection import train_test_split

In [29]:
singlishs_train, singlishs_test = train_test_split(singlishs, test_size = 2000)

In [32]:
import IPython.display as ipd
ipd.Audio('/home/husein/speech-bahasa/part3-splitted/wav/3211-1-0-2986.wav')

In [31]:
singlishs[-100:]

[('/home/husein/speech-bahasa/part3-splitted/wav/3247-2-0-2929.wav', 'yup'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3271-2-0-2277.wav',
  'half an hour'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3030-1-0-1919.wav',
  'if you go to China you will find that all the China'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3211-2-0-803.wav',
  '!wow! why are my question all so deep'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3112-2-0-3138.wav',
  'big mouth then he will tell the boss'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3122-2-0-33.wav', 'nothing'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3219-2-0-2107.wav',
  'I think I think is is mopping and cleaning'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3057-1-0-1301.wav',
  'on my surface pro is it does it use a lot of'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3163-2-0-1856.wav',
  'cabinets'),
 ('/home/husein/speech-bahasa/part3-splitted/wav/3141-2-0-457.wav',
  'how do you w

In [33]:
with open('singlish-mixed.json', 'w') as fopen:
    json.dump({'train': singlishs_train, 'test': singlishs_test}, fopen)

In [52]:
import IPython.display as ipd

ipd.Audio(audios[2])

In [54]:
audios = singlishs_train
audios, texts = zip(*audios)

In [55]:
len(audios), len(texts)

(1464521, 1464521)

In [56]:
texts[2]

"I don't have a certificate it'll be very difficult to find another job"

In [57]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    string = unicodedata.normalize('NFC', string).lower()
    string = string.replace('\'', '')
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [58]:
import mp

In [59]:
def loop(strings):
    strings, _ = strings
    results = []
    for i in tqdm(range(len(strings))):
        results.append((strings[i][0], preprocessing_text(strings[i][1])))
    return results

In [60]:
no_texts = [(i, texts[i]) for i in range(len(texts))]

In [61]:
loop((no_texts[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 18196.55it/s]


[(0, 'and the government has also come on board to support these efforts'),
 (1, 'thinking'),
 (2, 'i dont have a certificate itll be very difficult to find another job'),
 (3, 'but interesting if you marry into a different'),
 (4,
  'automated attendance task logging and emergency cleaning requests are also done through mobile'),
 (5, 'if you want me to'),
 (6, 'yes i think so'),
 (7, 'here are some of the more unusual and interesting courses'),
 (8, 'i'),
 (9, 'but')]

In [62]:
processed_text = mp.multiprocessing(no_texts, loop, cores = 20)

100%|██████████| 73226/73226 [00:02<00:00, 25081.09it/s]


In [63]:
len(processed_text)

1464521

In [64]:
audios[2]

'/home/husein/speech-bahasa/WAVE/SPEAKER0358/SESSION1/003581789.WAV'

In [65]:
processed_text_sorted = sorted(processed_text, key=lambda x: x[0])

In [66]:
processed_text_sorted[:10]

[(0, 'and the government has also come on board to support these efforts'),
 (1, 'thinking'),
 (2, 'i dont have a certificate itll be very difficult to find another job'),
 (3, 'but interesting if you marry into a different'),
 (4,
  'automated attendance task logging and emergency cleaning requests are also done through mobile'),
 (5, 'if you want me to'),
 (6, 'yes i think so'),
 (7, 'here are some of the more unusual and interesting courses'),
 (8, 'i'),
 (9, 'but')]

In [67]:
text_only = [i[1] for i in processed_text_sorted]
text_only[:10]

['and the government has also come on board to support these efforts',
 'thinking',
 'i dont have a certificate itll be very difficult to find another job',
 'but interesting if you marry into a different',
 'automated attendance task logging and emergency cleaning requests are also done through mobile',
 'if you want me to',
 'yes i think so',
 'here are some of the more unusual and interesting courses',
 'i',
 'but']

In [68]:
import malaya_speech

tokenizer = malaya_speech.subword.generate_tokenizer(text_only, max_subword_length = 3)
malaya_speech.subword.save(tokenizer, 'transducer-singlish.subword')

In [69]:
with open('bahasa-asr-train.json') as fopen:
    bahasa = json.load(fopen)
    
bahasa['X'][:1]

['/home/husein/speech-bahasa/cv-corpus-5.1-2020-06-22/id/clips/common_voice_id_20425643.mp3']

In [70]:
len(audios), len(text_only)

(1464521, 1464521)

In [71]:
audios = bahasa['X'] + list(audios)
processed_text = bahasa['Y'] + list(text_only)

In [72]:
with open('mixed-asr-train.json', 'w') as fopen:
    json.dump({'X': audios, 'Y':processed_text}, fopen)

In [73]:
tokenizer = malaya_speech.subword.generate_tokenizer(processed_text, max_subword_length = 3)

In [74]:
malaya_speech.subword.save(tokenizer, 'transducer-mixed.subword')

In [75]:
tokenizer = malaya_speech.subword.load('transducer-mixed.subword')
malaya_speech.subword.encode(tokenizer, 'i hate', add_blank = True)

[0, 2, 386, 860]

In [77]:
malaya_speech.subword.decode(tokenizer, [0, 2, 386, 860])

'i hate'

In [ ]:
# from pydub import AudioSegment
# import numpy as np

# sr = 16000

# def mp3_to_wav(file, sr = sr):
#     audio = AudioSegment.from_file(file)
#     audio = audio.set_frame_rate(sr).set_channels(1)
#     sample = np.array(audio.get_array_of_samples())
#     return malaya_speech.astype.int_to_float(sample), sr

# def generator(maxlen = 18, min_length_text = 2):
#     for i in tqdm(range(len(audios))):
#         try:
#             if audios[i].endswith('.mp3'):
#                 wav_data, _ = mp3_to_wav(audios[i])
#             else:
#                 wav_data, _ = malaya_speech.load(audios[i])
                
#             if (len(wav_data) / sr) > maxlen:
#                 # print(f'skipped audio too long {audios[i]}')
#                 continue
                
#             if len(processed_text[i]) < min_length_text:
#                 print(f'skipped text too short {audios[i]}')
#                 continue    

#             yield {
#                 'waveforms': wav_data.tolist(),
#                 'waveform_lens': [len(wav_data)],
#                 'targets': malaya_speech.subword.encode(tokenizer, processed_text[i], add_blank = False),
#             }
#         except Exception as e:
#             print(e)
            
# generator = generator()

In [ ]:
# import os
# import tensorflow as tf

# os.system('rm bahasa-asr/data/*')
# DATA_DIR = os.path.expanduser('bahasa-asr/data')
# tf.gfile.MakeDirs(DATA_DIR)

In [ ]:
# shards = [{'split': 'train', 'shards': 1000}]

In [ ]:
# import malaya_speech.train as train

# train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')